In [55]:
# Import libraries
import numpy as np
import pandas as pd
from scipy.linalg import block_diag
from collections import defaultdict
import random
import csv

In [56]:
# Define the parameters
data_path = "./cost_10_07.csv"
part_col = "Part"
dep_arr_site_col = "Departure/Arrival site"
arr_dep_site_col =  "Arrival/Departure site"
cost_col = "Cost"
PBS = [(2, 1), (3, 1), (4, 1), (5, 2) , (6, 2), (7, 2), (8, 3), (9, 4), (10, 4)]
subpart = [(2,3),(2,4),(3,4),(5,6),(5,7),(6,7),(9,10)]
lambda_1 = 100
lambda_2 = 100
lambda_3 = 100

In [57]:
def cost_matrix_df(data_path, part_col, dep_arr_site_col, arr_dep_site_col, cost_col):
    # Load the data and create a DataFrame
    data = pd.read_csv(data_path)
    data_df = pd.DataFrame(data)

    # Create the pivot DataFrame
    part_range = range(2, data_df[part_col].max() + 1)  # Start from 2
    site_range = range(1, max(data_df[dep_arr_site_col].max(), data_df[arr_dep_site_col].max()) + 1)

    pivot_df = (
        pd.DataFrame(
            index=pd.MultiIndex.from_product(
                [part_range, site_range, site_range], 
                names=['Part', 'Departure_Site', 'Arrival_Site']
            )
        )
        .reset_index()
        .merge(data_df.assign(Part_Column=data_df[part_col]), how='left', left_on=['Part', 'Departure_Site', 'Arrival_Site'], right_on=[part_col, dep_arr_site_col, arr_dep_site_col])
        .assign(Cost=lambda df: df[cost_col].fillna(0))
        .pivot_table(index=['Part', 'Departure_Site'], columns=['Part_Column', 'Arrival_Site'], values='Cost', fill_value=0)
        .pipe(lambda df: df.reorder_levels(['Part_Column', 'Arrival_Site'], axis=1))
        .pipe(lambda df: df.reindex(columns=pd.MultiIndex.from_product([part_range, site_range], names=df.columns.names), fill_value=0))
        .pipe(lambda df: df.reindex(index=pd.MultiIndex.from_product([part_range, site_range], names=df.index.names), fill_value=0))
        .pipe(lambda df: df + df.T)  # Make the DataFrame symmetric
        .pipe(lambda df: df.where(np.eye(df.shape[0], dtype=bool), df / 2))  # Halve the off-diagonal elements and keep the diagonal the same
    )

    return pivot_df

In [58]:
cost_matrix_df(data_path, part_col, dep_arr_site_col, arr_dep_site_col, cost_col)

Part_Column             2                                              3   \
Arrival_Site             1      2      3      4      5      6      7    1   
Part Departure_Site                                                         
2    1               0.000  0.820  0.525  0.545  0.715  0.455  0.850  0.0   
     2               0.820  0.000  0.295  0.185  0.560  0.120  0.895  0.0   
     3               0.525  0.295  0.000  0.465  0.510  0.675  0.825  0.0   
     4               0.545  0.185  0.465  0.000  0.825  0.095  0.760  0.0   
     5               0.715  0.560  0.510  0.825  0.000  0.520  0.350  0.0   
...                    ...    ...    ...    ...    ...    ...    ...  ...   
10   3               0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.0   
     4               0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.0   
     5               0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.0   
     6               0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.0   
     7               0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.0   

Part_Column                    ...   9                10                      \
Arrival_Site           2    3  ...    5    6    7      1      2     3      4   
Part Departure_Site            ...                                             
2    1               0.0  0.0  ...  0.0  0.0  0.0  0.000  0.000  0.00  0.000   
     2               0.0  0.0  ...  0.0  0.0  0.0  0.000  0.000  0.00  0.000   
     3               0.0  0.0  ...  0.0  0.0  0.0  0.000  0.000  0.00  0.000   
     4               0.0  0.0  ...  0.0  0.0  0.0  0.000  0.000  0.00  0.000   
     5               0.0  0.0  ...  0.0  0.0  0.0  0.000  0.000  0.00  0.000   
...                  ...  ...  ...  ...  ...  ...    ...    ...   ...    ...   
10   3               0.0  0.0  ...  0.0  0.0  0.0  1.560  0.875  0.00  1.390   
     4               0.0  0.0  ...  0.0  0.0  0.0  1.625  0.545  1.39  0.000   
     5               0.0  0.0  ...  0.0  0.0  0.0  2.135  1.670  1.51  2.465   
     6               0.0  0.0  ...  0.0  0.0  0.0  1.360  0.360  2.01  0.285   
     7               0.0  0.0  ...  0.0  0.0  0.0  2.525  2.660  2.45  2.260   

Part_Column                               
Arrival_Site             5      6      7  
Part Departure_Site                       
2    1               0.000  0.000  0.000  
     2               0.000  0.000  0.000  
     3               0.000  0.000  0.000  
     4               0.000  0.000  0.000  
     5               0.000  0.000  0.000  
...                    ...    ...    ...  
10   3               1.510  2.010  2.450  
     4               2.465  0.285  2.260  
     5               0.000  1.550  1.035  
     6               1.550  0.000  2.335  
     7               1.035  2.335  0.000  

[63 rows x 63 columns]

In [59]:
# Define the function to create the cost function
def cost_function_qubo(PBS):
    # Get the range of parts and sites
    pivot_df = cost_matrix_df(data_path, part_col, dep_arr_site_col, arr_dep_site_col, cost_col)
    site_range = pivot_df.index.get_level_values('Departure_Site').unique()

    # Convert the pivot DataFrame to a QUBO dictionary
    cost_function = {((r, i), (s, j)): pivot_df.loc[(r, i), (r, j)] 
            for r, s in PBS for i in site_range for j in site_range if i != j and r != s and (r, i) in pivot_df.index and (r, j) in pivot_df.index}

    return cost_function

In [60]:
# Define the function to create the first constraint function
def penalty_C1_qubo(PBS, penalty=1000):
    
    data = pd.read_csv(data_path)
    data_df = pd.DataFrame(data)
    # Determine the maximum number of sites
    max_sites = data_df['Arrival/Departure site'].max()

    # Create a QUBO dictionary to hold the penalty terms
    penalty_qubo_C1 = {}
    for r, _ in PBS:
        for i in range(1, max_sites + 1):
            penalty_qubo_C1[((r, i), (r, i))] = -penalty
            for j in range(i + 1, max_sites + 1):
                penalty_qubo_C1[((r, i), (r, j))] = 2 * penalty
                penalty_qubo_C1[((r, j), (r, i))] = 2* penalty

    return penalty_qubo_C1

In [61]:
# Define the function to create the second constraint function
def penalty_C2_qubo(PBS, penalty=1000):
    
    data = pd.read_csv(data_path)
    data_df = pd.DataFrame(data)
    # Determine the maximum number of sites
    max_sites = data_df['Arrival/Departure site'].max()

    # Create a QUBO dictionary to hold the penalty terms
    penalty_qubo_C2 = {}
    for r, s in PBS:
        for i in range(1, max_sites + 1):
            # Add a penalty if the sum of Xr,iXs,i is non-zero
            penalty_qubo_C2[((r, i), (s, i))] = penalty

    return penalty_qubo_C2

In [62]:
# Define the function to create the third constraint function
def penalty_C3_qubo(subpart, penalty=1000):
    
    data = pd.read_csv(data_path)
    data_df = pd.DataFrame(data)
    # Determine the maximum number of sites
    max_sites = data_df['Arrival/Departure site'].max()


    # Create a QUBO dictionary to hold the penalty terms
    penalty_qubo_C3 = {}
    for r, s in subpart:
        if r < s:
            for i in range(1, max_sites + 1):
                # Add a penalty if the origins of two subparts of a common part are the same
                penalty_qubo_C3[((r, i), (s, i))] = penalty

    return penalty_qubo_C3

In [63]:
# Auxiliary function to multiply penalties by QUBO dictionaries

def multiply_penalty_qubo(penalty, qubo_dict):
    # Create a new QUBO dictionary to hold the result
    result_qubo = {}

    # Multiply each value in the QUBO dictionary by the penalty
    for key, value in qubo_dict.items():
        result_qubo[key] = penalty * value

    return result_qubo

In [64]:
# Auxiliary function to add QUBO dictionaries
def add_qubo_dicts(qubo_dict1, qubo_dict2):
    # Create a new defaultdict to hold the result
    result_qubo = defaultdict(int)

    # Add the values from the first QUBO dictionary
    for key, value in qubo_dict1.items():
        result_qubo[key] += value

    # Add the values from the second QUBO dictionary
    for key, value in qubo_dict2.items():
        result_qubo[key] += value

    return dict(result_qubo)

In [65]:
C = cost_function_qubo(PBS)
C1 = penalty_C1_qubo(PBS)
C2 = penalty_C2_qubo(PBS)
C3 = penalty_C3_qubo(subpart)

In [68]:
# List of tuples (penalty, QUBO dictionary)
penalties_qubos = [(1, C), (lambda_1, C1), (lambda_2, C2), (lambda_3, C3)]

# Initialize Q as an empty dictionary
Q = {}

# Iterate over the list of tuples
for penalty, qubo in penalties_qubos:
    # Multiply the QUBO dictionary by its penalty
    qubo_multiplied = multiply_penalty_qubo(penalty, qubo)
    
    # Add the multiplied QUBO dictionary to Q
    Q = add_qubo_dicts(Q, qubo_multiplied)

In [73]:
C2

{((2, 1), (1, 1)): 1000,
 ((2, 2), (1, 2)): 1000,
 ((2, 3), (1, 3)): 1000,
 ((2, 4), (1, 4)): 1000,
 ((2, 5), (1, 5)): 1000,
 ((2, 6), (1, 6)): 1000,
 ((2, 7), (1, 7)): 1000,
 ((3, 1), (1, 1)): 1000,
 ((3, 2), (1, 2)): 1000,
 ((3, 3), (1, 3)): 1000,
 ((3, 4), (1, 4)): 1000,
 ((3, 5), (1, 5)): 1000,
 ((3, 6), (1, 6)): 1000,
 ((3, 7), (1, 7)): 1000,
 ((4, 1), (1, 1)): 1000,
 ((4, 2), (1, 2)): 1000,
 ((4, 3), (1, 3)): 1000,
 ((4, 4), (1, 4)): 1000,
 ((4, 5), (1, 5)): 1000,
 ((4, 6), (1, 6)): 1000,
 ((4, 7), (1, 7)): 1000,
 ((5, 1), (2, 1)): 1000,
 ((5, 2), (2, 2)): 1000,
 ((5, 3), (2, 3)): 1000,
 ((5, 4), (2, 4)): 1000,
 ((5, 5), (2, 5)): 1000,
 ((5, 6), (2, 6)): 1000,
 ((5, 7), (2, 7)): 1000,
 ((6, 1), (2, 1)): 1000,
 ((6, 2), (2, 2)): 1000,
 ((6, 3), (2, 3)): 1000,
 ((6, 4), (2, 4)): 1000,
 ((6, 5), (2, 5)): 1000,
 ((6, 6), (2, 6)): 1000,
 ((6, 7), (2, 7)): 1000,
 ((7, 1), (2, 1)): 1000,
 ((7, 2), (2, 2)): 1000,
 ((7, 3), (2, 3)): 1000,
 ((7, 4), (2, 4)): 1000,
 ((7, 5), (2, 5)): 1000,
